In [1]:
from transformers import AutoTokenizer
import transformers
import torch

In [2]:
model_name = "codellama/CodeLlama-7b-hf"
model_dir = "./CodeLlama-7b-Python-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
pipeline = transformers.pipeline(
    "text-generation",
    model=model_dir,
    torch_dtype=torch.float16,
    device_map="cpu",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
def AI_response_ended(txt):
    n = len(txt)
    if n == 1:
        return False
    if n == 2 and txt[1].startswith("{{AI}}"): 
        return True
    for sentance in range (2, n):
        if txt[sentance].startswith("{{user}}"):
            return True
    return False

In [3]:
CONTEXT = "You are an API to convert user\u2019s text description of 3D objects into python code that can be used to generate 3D objects in Blender. You have access to a library of pre-defined 3D objects, (cups, knives, spoons, forks, and dishes) and you can generate 3D objects by combining these objects. You can also generate 3D objects by modifying the pre-defined objects (e.g., changing the size of the object, changing the color of the object, etc.).\n{{user}}: make a fork\n{{AI}}: forkObj=fork.ForkFactory(0).spwan_asset()\n{{user}}: change color of it to [red|green|blue]\n{{AI}}: forkObj.color = [(255,0,0,255)|(0,255,0,255)|(0,0,255,255)] #(r,g,b,a)\n{{user}}: make it [small | medium | big]\n{{AI}}: forkObj.dimensions = [(forkObj.dimensions[0] / 2, forkObj.dimensions[1] / 2, forkObj.dimensions[2] / 2)| (forkObj.dimensions[0] * 1.5, forkObj.dimensions[1] * 1.5, forkObj.dimensions[2] * 1.5)|(forkObj.dimensions[0] * 2),forkObj.dimensions[1] * 2), forkObj.dimensions[2] * 2)]\n{{user}}: make a horn\n{{AI}}: **Sorry, I can Only make cups, spoons, knives, forks, and dishes**\n{{user}}: make 3 cups\n{{AI}}: cupObjs = [cup.CupFactory(0).spawn_asset() for i in range(3)]\n{{user}}: make 1 red, 1 blue and 1 green\n{{AI}}: cupObjs[0].color = (255,0,0,255)\ncupObjs[1].color = (0,255,0,255)\ncupObjs[2].color = (0,0,255,255)\n{{user}}: move one to the left, one to the right, one to the top\n{{AI}}: cupObjs[0].location[0] += cupObjs[0].dimensions[0] * 2\ncupObjs[1].location[0] -= cupObjs[1].dimensions[0] * 2\ncupObjs[2].location[1] += cupObjs[2].dimensions[1] * 2"

def get_last_ai_response(conversation):
    # Find the last occurrence of "{{AI}}: " and "{{user}}"
    pos1 = conversation.rfind("{{AI}}: ")
    pos2 = conversation.rfind("{{user}}")

    if pos1 == -1:
        return None
    new_convo = conversation[pos1:pos2]
    return new_convo


def talk_with_ai(text, old_conv_len, max_new_tokens = 30):
    old_max_new_tokens = max_new_tokens
    sequences = pipeline(
    text,
    do_sample=True,
    temperature=0.29,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=max_new_tokens)

    new_lines = sequences[0]['generated_text'][old_conv_len:].split("\n")
    
    # # check the last line if it starts with ...
    # if (not AI_response_ended("{{user}}")):
    #     return talk_with_ai(text, old_conv_len, max_new_tokens = max_new_tokens + 10)
    # #print(new_lines)
    
    return get_last_ai_response("\n".join(new_lines))


def chat(text, conversation):
    old_conv_len = len(conversation)
    conversation += "{{user}}: " + text + "\n"
    conversation += "{{AI}}: "
    ai_response = talk_with_ai(conversation, old_conv_len)
    conversation += ai_response
    return ai_response
    
    

In [4]:
conversation = CONTEXT
print(chat("make a knive", conversation))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


KeyboardInterrupt: 

In [52]:

print(chat("change the color of it to yellow", conversation))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['{{user}}: change the color of it to yellow', '{{AI}}:  spoonObj.color = (255,255,0,255)', '{{user}}: make a cup', '']
{{AI}}:  spoonObj.color = (255,255,0,255)



In [257]:
print(chat("make it big",conversation))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['{{user}}: make it big', '{{AI}}:  forkObj.dimensions = (forkObj.dimensions[0] * 2, forkObj.dimensions[1] * ']
{{AI}}:  forkObj.dimensions = (forkObj.dimensions[0] * 2, forkObj.dimensions[1] * 2, forkObj.dimensions[2] * 2)



In [ ]:
!nvidia-smi